In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![View on GitHub][github-badge]][github-basic] [![Open In Colab][colab-badge]][colab-basic] [![Open in Binder][binder-badge]][binder-basic]

[github-badge]: https://img.shields.io/badge/View-on%20GitHub-blue?logo=GitHub
[colab-badge]: https://colab.research.google.com/assets/colab-badge.svg
[binder-badge]: https://static.mybinder.org/badge_logo.svg

[github-basic]: Basic_VGG_in_Keras.ipynb
[colab-basic]: https://colab.research.google.com/github/mbrukman/reimplementing-ml-papers/blob/main/vgg/Basic_VGG_in_Keras.ipynb
[binder-basic]: https://mybinder.org/v2/gh/mbrukman/reimplementing-ml-papers/main?filepath=vgg/Basic_VGG_in_Keras.ipynb

In [ ]:
from tensorflow import keras
from keras import Input, Sequential
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPool2D

In [ ]:
!curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/alexnet/local_response_normalization.py

from local_response_normalization import LocalResponseNormalization

In [ ]:
def Conv(filters: int, kernel_size: int, **kwargs) -> Conv2D:
    """Shorthand for defining the Conv2D layers for VGG family of models.
    
    All VGG models have a stride of 1 and 'same' padding, so to avoid repeating
    these parameters (or skipping them, which leads to 'valid' padding), we use
    a convenience function here.

    Additionally, all hidden layers in VGG are specified to use ReLU activation,
    so we include that here as well.
    """
    return Conv2D(filters, kernel_size, strides=(1, 1), padding='same',
                  activation='relu', **kwargs)


def MaxPool(**kwargs) -> MaxPool2D:
    """Shorthand for defining the MaxPool layers for VGG fmaily of models.
    
    All pooling layers in VGG are using kernel size of 2 with a stride of 2, so
    we define a shorthand here to ensure their uniformity.
    """
    return MaxPool2D(pool_size=(2, 2), strides=(2, 2), **kwargs)

In [ ]:
# Available model types.
MODEL_A = 'A'
MODEL_A_LRN = 'A-LRN'
MODEL_B = 'B'
MODEL_C = 'C'
MODEL_D = 'D'
MODEL_E = 'E'

In [ ]:
def VGG(model: str) -> Sequential:
    """Defines a specific VGG model, given one of the valid model types."""
    assert model in (MODEL_A, MODEL_A_LRN, MODEL_B, MODEL_C, MODEL_D, MODEL_E)

    vgg = Sequential([
        Input(shape=(224, 224, 3)),
    ], name=f'VGG-{model}')

    # First block
    vgg.add(Conv2D(64, 3, 1, name='Conv2D_1_1'))
    if model == MODEL_A:
        # No other layers are added here.
        pass
    elif model == MODEL_A_LRN:
        vgg.add(LocalResponseNormalization(name='LRN'))
    else:
        vgg.add(Conv(64, 3, name='Conv2D_1_2'))
    
    vgg.add(MaxPool(name='MaxPool_1'))

    # Second block
    vgg.add(Conv(128, 3, name='Conv2D_2_1'))
    if model in (MODEL_B, MODEL_C, MODEL_D, MODEL_E):
        vgg.add(Conv(128, 3, name='Conv2D_2_2'))

    vgg.add(MaxPool(name='MaxPool_2'))

    # Third block
    vgg.add(Conv(256, 3, name='Conv2D_3_1'))
    vgg.add(Conv(256, 3, name='Conv2D_3_2'))

    if model == MODEL_C:
        vgg.add(Conv(256, 1, name='Conv2D_3_3'))
    elif model in (MODEL_D, MODEL_E):
        vgg.add(Conv(256, 3, name='Conv2D_3_3'))

    # Model E gets an extra layer.
    if model == MODEL_E:
        vgg.add(Conv(256, 3, name='Conv2D_3_4'))

    vgg.add(MaxPool(name='MaxPool_3'))

    # Fourth block
    vgg.add(Conv(512, 3, name='Conv2D_4_1'))
    vgg.add(Conv(512, 3, name='Conv2D_4_2'))

    if model == MODEL_C:
        vgg.add(Conv(512, 1, name='Conv2D_4_3'))
    elif model in (MODEL_D, MODEL_E):
        vgg.add(Conv(512, 3, name='Conv2D_4_4'))

    # Model E gets an extra layer.
    if model == MODEL_E:
        vgg.add(Conv(512, 3, name='Conv2D_4_5'))

    vgg.add(MaxPool(name='MaxPool_4'))

    # Fifth block
    vgg.add(Conv(512, 3, name='Conv2D_5_1'))
    vgg.add(Conv(512, 3, name='Conv2D_5_2'))
    if model == MODEL_C:
        vgg.add(Conv(512, 1, name='Conv2D_5_3'))
    elif model in (MODEL_D, MODEL_E):
        vgg.add(Conv(512, 3, name='Conv2D_5_3'))

    # Model E gets an extra layer.
    if model == MODEL_E:
        vgg.add(Conv(512, 3, name='Conv2D_5_4'))

    vgg.add(MaxPool(name='MaxPool_5'))

    vgg.add(Flatten(name='Flatten'))
    vgg.add(Dense(4096, name='FC_1', activation='relu'))
    vgg.add(Dense(4096, name='FC_2', activation='relu'))
    vgg.add(Dense(1000, name='FC_3', activation='relu'))
    vgg.add(Activation(keras.activations.softmax, name='Softmax'))

    return vgg

In [ ]:
VGG_A = VGG(MODEL_A)
VGG_A_LRN = VGG(MODEL_A_LRN)
VGG_B = VGG(MODEL_B)
VGG_C = VGG(MODEL_C)
VGG_D = VGG(MODEL_D)
VGG_E = VGG(MODEL_E)

In [ ]:
VGG_A.summary()

Model: "VGG-A"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168    
                                                                 
 Conv2D_3_2 (Conv2D)         (None, 55, 55, 256)       590080    
                                                                 
 MaxPool_3 (MaxPooling2D)    (None, 27, 27, 256)       0     

In [ ]:
VGG_A_LRN.summary()

Model: "VGG-A-LRN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 LRN (LocalResponseNormaliza  (None, 222, 222, 64)     0         
 tion)                                                           
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168    
                                                         

In [ ]:
VGG_B.summary()

Model: "VGG-B"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 Conv2D_1_2 (Conv2D)         (None, 222, 222, 64)      36928     
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 Conv2D_2_2 (Conv2D)         (None, 111, 111, 128)     147584    
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168

In [ ]:
VGG_C.summary()

Model: "VGG-C"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 Conv2D_1_2 (Conv2D)         (None, 222, 222, 64)      36928     
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 Conv2D_2_2 (Conv2D)         (None, 111, 111, 128)     147584    
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168

In [ ]:
VGG_D.summary()

Model: "VGG-D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 Conv2D_1_2 (Conv2D)         (None, 222, 222, 64)      36928     
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 Conv2D_2_2 (Conv2D)         (None, 111, 111, 128)     147584    
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168

In [ ]:
VGG_E.summary()

Model: "VGG-E"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1_1 (Conv2D)         (None, 222, 222, 64)      1792      
                                                                 
 Conv2D_1_2 (Conv2D)         (None, 222, 222, 64)      36928     
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 111, 111, 64)      0         
                                                                 
 Conv2D_2_1 (Conv2D)         (None, 111, 111, 128)     73856     
                                                                 
 Conv2D_2_2 (Conv2D)         (None, 111, 111, 128)     147584    
                                                                 
 MaxPool_2 (MaxPooling2D)    (None, 55, 55, 128)       0         
                                                                 
 Conv2D_3_1 (Conv2D)         (None, 55, 55, 256)       295168